## Start Clipper

In [34]:
import sys
import os
sys.path.append(os.path.abspath('../../management/'))
import clipper_manager as cm
reload(cm)
user = ""
key = ""
host = "localhost"
clipper = cm.Clipper(host, user, key)

Checking if Docker is running...


In [35]:
clipper.start()

Clipper is running


In [36]:
app_name = "example_app"
model_name = "example_model_name"
default_output = "0"
input_type = "doubles"

clipper.register_application(
    app_name,
    model_name,
    input_type,
    default_output,
    slo_micros=20000)

Error application example_app already exists


In [37]:
clipper.get_all_apps(verbose=True)

[{u'candidate_model_names': [u'example_model_name'],
  u'default_output': u'0',
  u'input_type': u'doubles',
  u'latency_slo_micros': 20000,
  u'name': u'example_app'},
 {u'candidate_model_names': [u'birds_vs_planes_classifier'],
  u'default_output': u'0',
  u'input_type': u'doubles',
  u'latency_slo_micros': 20000,
  u'name': u'cifar_demo'}]

## Define prediction function

In [38]:
def noop_predict(inputs):
    outputs = []
    for input_item in inputs:
        outputs.append(str(sum(input_item)))
    return outputs

In [39]:
noop_predict([[3,3], [4,54]])

['6', '58']

In [40]:
clipper.deploy_predict_function(
    model_name,
    8,
    noop_predict,
    ["label1", "label2"],
    input_type,
)

Fetching package metadata .........
Solving package specifications: .
Solving package specifications: ..........
The following packages in your conda environment aren't available in the linux-64 conda channel the container will use:
appnope 0.1.0 py27_0, libgfortran 3.0.0 0
We will skip their installation when deploying your function. If your function uses these packages, the container will experience a runtime error when queried.


Supplied environment details
Serialized and supplied predict function
Published model to Clipper
Found nishadsingh/predict_func_container:latest on host
Copied model data to host


True

In [43]:
import json
import requests
def predict(x):
    uid = 0
    url = "http://%s:1337/%s/predict" % (host, app_name)
    req_json = json.dumps({'uid': uid, 'input': list(x)})
    headers = {'Content-type': 'application/json'}
    r = requests.post(url, headers=headers, data=req_json)
    return r.text

In [45]:
predict([2.0, 3.0])

u'{"query_id":0,"output":5.0,"default":false}'